In [ ]:
"""

Prepared clean stereographic grid for NEMO runs (5 kms) with bounds and so on

"""

In [ ]:
import xarray as xr
from cdo import Cdo
import multimelt.useful_functions as uf
from tqdm.notebook import tqdm

In [ ]:
outputpath = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/'

In [ ]:
ISMIP_4km_grid = xr.open_dataset(outputpath + 'ISMIP6_AIS_4000m_grid.nc')

In [ ]:
ISMIP_4km_grid

In [ ]:
ISMIP_4km_grid['cell_area'].plot()

In [ ]:
uf.change_coord_stereo_to_latlon(ISMIP_4km_grid.x[100] - 2000.,ISMIP_4km_grid.y[110] + 2000.)

In [ ]:
ISMIP_4km_grid['mapping'].load()#['lon_bnds'].isel(x=100,y=110,nv4=3).load()

In [ ]:
nemo_5km_grid = xr.open_dataset(outputpath+'NEMO_ctrl94_ANT_STEREO/stereo_grid.nc')

In [ ]:
half_res = 2500.

In [ ]:
meshx,meshy = np.meshgrid(nemo_5km_grid.x,nemo_5km_grid.y)

In [ ]:
meshlon_ll,meshlat_ll = uf.change_coord_stereo_to_latlon(meshx - half_res, meshy - half_res)
meshlon_lr,meshlat_lr = uf.change_coord_stereo_to_latlon(meshx + half_res, meshy - half_res)
meshlon_ur,meshlat_ur = uf.change_coord_stereo_to_latlon(meshx + half_res, meshy + half_res)
meshlon_ul,meshlat_ul = uf.change_coord_stereo_to_latlon(meshx - half_res, meshy + half_res)

In [ ]:
meshlon,meshlat = uf.change_coord_stereo_to_latlon(meshx, meshy)

In [ ]:
lon_bnds = xr.DataArray(data=np.array([meshlon_ll, meshlon_lr, meshlon_ur, meshlon_ul]), dims=['nv4','y','x'])
lat_bnds = xr.DataArray(data=np.array([meshlat_ll, meshlat_lr, meshlat_ur, meshlat_ul]), dims=['nv4','y','x'])

In [ ]:
lon = xr.DataArray(data=np.array(meshlon), dims=['y','x'])
lat = xr.DataArray(data=np.array(meshlat), dims=['y','x'])

In [ ]:
nemo_5km_new_grid = xr.merge([lon_bnds.rename('lon_bnds'),lat_bnds.rename('lat_bnds')]).assign_coords({'lon': lon, 'lat': lat, 'y': nemo_5km_grid.y, 'x': nemo_5km_grid.x})

In [ ]:
nemo_5km_new_grid['mapping'] = ISMIP_4km_grid['mapping'].copy()

In [ ]:
nemo_5km_new_grid.transpose('y','x','nv4').to_netcdf(outputpath + 'nemo_5km_stereo_withbnds.nc')

In [ ]:
nemo_5km_new_grid[['lon','lat']].to_netcdf(outputpath + 'nemo_5km_stereo_mapping.nc')

In [ ]:
gridinfo = open(outputpath+"griddes_4000m.txt", "r")

In [ ]:
with open(outputpath+'griddes_4000m.txt','r') as file:
    line_list = file.readlines()
    print(ll)

In [ ]:
for ll in range(len(line_list)):
    if 'bounds' in line_list[ll][0:10]:
        print(line_list[ll])
        print(ll)

In [ ]:
meshlat_lr[::-1]

In [ ]:
for ll in range(289018,289024):
    print(line_list[ll])
    

In [ ]:
nemo_5km_new_grid.transpose('y','x','nv4')

In [ ]:
ISMIP_4km_grid

In [ ]:
with open(outputpath+"stereo_grid_curvilinear_definition.txt", "w") as txt_file:

    txt_file.write('# \n')
    txt_file.write('# gridID 1 \n')
    txt_file.write('# \n')
    txt_file.write('gridtype  = curvilinear \n')
    txt_file.write('gridsize  = 1779556 \n')
    txt_file.write('datatype  = float \n')
    txt_file.write('xsize     = 1334 \n')
    txt_file.write('ysize     = 1334 \n')
    txt_file.write('xname     = lon \n')
    txt_file.write('xdimname  = x \n')
    txt_file.write('xlongname = "longitude" \n')
    txt_file.write('xunits    = "degrees_east" \n')
    txt_file.write('yname     = lat \n')
    txt_file.write('ydimname  = y \n')
    txt_file.write('ylongname = "latitude" \n')
    txt_file.write('yunits    = "degrees_north" \n')
    txt_file.write('vdimname  = nv4 \n')
    txt_file.write('xvals = ')
    for vv in tqdm(meshlon.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('yvals = ')
    for vv in tqdm(meshlat.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('xbounds = ')
    for vv in tqdm(nemo_5km_new_grid['lon_bnds'].values.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('ybounds = ')
    for vv in tqdm(nemo_5km_new_grid['lat_bnds'].values.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('gridtype = projection \n')
    txt_file.write('gridsize  = 1779556 \n')
    txt_file.write('xsize     = 1334 \n')
    txt_file.write('ysize     = 1334 \n')
    txt_file.write('xname     = x \n')
    txt_file.write('xlongname = "x-coordinate in Cartesian system" \n')
    txt_file.write('xunits = "m" \n')
    txt_file.write('yname     = y \n')
    txt_file.write('ylongname = "y-coordinate in Cartesian system" \n')
    txt_file.write('yunits = "m" \n')
    txt_file.write('xfirst = '+str(nemo_5km_new_grid.x.min().values)+' \n')
    txt_file.write('xinc = 5000 \n')
    txt_file.write('yfirst = '+str(nemo_5km_new_grid.y.min().values)+' \n')
    txt_file.write('yinc = 5000 \n')
    txt_file.write('grid_mapping = Projection \n')
    
txt_file.close()